Writing this code for eyes closed / open detection

In [1]:
# libraries
import cv2
import numpy as np
import os
import pandas as pd
import dlib

In [6]:
# pip install opencv-python

In [10]:
!pip install cmake

In [16]:
!pip install dlib

  Using cached dlib-19.24.2.tar.gz (11.8 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
Failed to build dlib


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\karnb\anaconda3\python.exe' 'C:\Users\karnb\anaconda3\lib\site-packages\pip\_vendor\pep517\in_process\_in_process.py' build_wheel 'C:\Users\karnb\AppData\Local\Temp\tmpaceueejf'
       cwd: C:\Users\karnb\AppData\Local\Temp\pip-install-xluuyqbi\dlib_c40ab8ba9a3b4033b12ff7d1e162f7d0
  Complete output (73 lines):
  running bdist_wheel
  running build
  running build_ext
  <string>:125: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  Building extension for Python 3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]
  Invoking CMake setup: 'cmake C:\Users\karnb\AppData\Local\Temp\pip-install-xluuyqbi\dlib_c40ab8ba9a3b4033b12ff7d1e162f7d0\tools\python -DCMAKE_LIBRARY_OUTPUT_DIRECTORY=C:\Users\karnb\AppData\Local\Temp\pip-install-xluuyqbi\dlib_c40ab8ba9a3b4033b12ff7d1e162f7d0\build\lib.win-amd64-cpython-39 -DPYTHON_EXECUTABLE=C:\Users\karnb\anaconda3\python.exe -D

In [9]:
# !pip install imutils

  Using cached imutils-0.5.4.tar.gz (17 kB)
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25872 sha256=5b355095d96ff2e3c72286a51fd71b1986055511bb4493642794ee57d94d09ca
  Stored in directory: c:\users\karnb\appdata\local\pip\cache\wheels\4b\a5\2d\4a070a801d3a3d93f033d3ee9728f470f514826e89952df3ea
Successfully built imutils


In [3]:
# pip install pandas

In [2]:
import imutils
from imutils import face_utils
from scipy.spatial import distance

In [3]:
detect=dlib.get_frontal_face_detector()
predict=dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [4]:
def eye_aspect_ratio(eye):
    A=distance.euclidean(eye[1],eye[5])
    B=distance.euclidean(eye[2],eye[4])
    C=distance.euclidean(eye[0],eye[3])
    ear=(A+B)/(2.0*C)
    return ear

In [5]:
(lstart,lend)=face_utils.FACIAL_LANDMARKS_68_IDXS['left_eye']
(rstart,rend)=face_utils.FACIAL_LANDMARKS_68_IDXS['right_eye']

In [6]:
def lip_distance(shape):
    toplip=shape[50:53]
    toplip=np.concatenate((toplip,shape[61:64]))

    lowlip=shape[56:59]
    lowlip=np.concatenate((lowlip,shape[65:68]))

    topmean=np.mean(toplip,axis=0)
    lowmean=np.mean(lowlip,axis=0)
    distance=abs(topmean[1]-lowmean[1])
    return distance

In [7]:
ear_thresh=0.25
yawn_thresh=20
flag=0
frame_check=25

In [8]:
cap=cv2.VideoCapture(0)

while True:
    ret,frame=cap.read()
    frame=imutils.resize(frame,width=450)
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    subjects=detect(gray,0)
    for subject in subjects:
        shape=predict(gray,subject)
        shape=face_utils.shape_to_np(shape)
        lefteye=shape[lstart:lend]
        righteye=shape[rstart:rend]
        lear=eye_aspect_ratio(lefteye)
        rear=eye_aspect_ratio(righteye)
        ear=(lear+rear)/2.0
        lefteyehull=cv2.convexHull(lefteye)
        righteyehull=cv2.convexHull(righteye)
        cv2.drawContours(frame,[lefteyehull],-1,(0,255,0),1)
        cv2.drawContours(frame,[righteyehull],-1,(0,255,0),1)

        distancelip=lip_distance(shape)
        lip = shape[48:60]
        cv2.drawContours(frame, [lip], -1, (0, 255, 0), 1)

        
        if (ear<ear_thresh):
            flag+=1
            print(flag)
            if (flag>=frame_check):
                cv2.putText(frame,"EYE CLOSE ALERT",(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
        else:
            flag=0

        if (distancelip>yawn_thresh):
            cv2.putText(frame,"YAWN ALERT",(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
            
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "YAWN: {:.2f}".format(distancelip), (300, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
    cv2.imshow("Frame", frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()
cap.release()

1
1
2
3
1
2
3
4
1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
1
2
1
2
3
1
2
3
4
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
1
2
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
1
2
1
2
3
4
5
1
2
3
4
5
6
7
8
9
1
2
3
4
5
1
2
3
4
5
6
7
8
1
2
3
4
1
2
1
2
3
4
1
2
3
4
5
1
2
3
1
2
3
4
1
1
2
3
4
5
6


In [10]:
cv2.destroyAllWindows()
cap.release()